In [1]:
import psycopg2

#### Database connection settings

In [2]:
db_settings = {
    'flight': {
        'host': "localhost",
        'port': 5001,
        'database': "postgres",
        'user': "postgres",
        'password': "pgpass"
    },
    'hotel': {
        'host': "localhost",
        'port': 5002,
        'database': "postgres",
        'user': "postgres",
        'password': "pgpass"
    },
    'account': {
        'host': "localhost",
        'port': 5003,
        'database': "postgres",
        'user': "postgres",
        'password': "pgpass"
    },
}

#### Initialization

In [3]:
db_connections = {
    'flight': psycopg2.connect(**db_settings['flight']),
    'hotel': psycopg2.connect(**db_settings['hotel']),
    'account': psycopg2.connect(**db_settings['account']),
}

for item in db_connections.items():
    print(item, f'version: {item[1].server_version}')

('flight', <connection object at 0x00000215323F4AE0; dsn: 'user=postgres password=xxx dbname=postgres host=localhost port=5001', closed: 0>) version: 110014
('hotel', <connection object at 0x00000215323F4BF0; dsn: 'user=postgres password=xxx dbname=postgres host=localhost port=5002', closed: 0>) version: 110014
('account', <connection object at 0x00000215323F4D00; dsn: 'user=postgres password=xxx dbname=postgres host=localhost port=5003', closed: 0>) version: 110014


In [4]:
query = '''
drop table if exists flight_booking;
create table flight_booking(
    booking_id serial primary key,
    client_name varchar(255),
    flight_number  varchar(255),   -- SCD1 attribure
    airpot_of_departure varchar(255),      -- SCD2 attribure
    destination_airport varchar(255), -- SCD2 attribure
    date int
);
truncate table flight_booking;
'''
with db_connections['flight'].cursor() as cursor:
    try:
        cursor.execute(query)
        db_connections['flight'].commit()
        print('flight_booking tables created in flight database')
    except Exception as e:
        print(e)
        db_connections['flight'].rollback()

flight_booking tables created in flight database


In [5]:
query = '''
drop table if exists hotel_booking;
create table hotel_booking(
    booking_id serial primary key,
    client_name varchar(255),
    hotel_name  varchar(255),
    check_in_date int,   
    check_out_date int
);
truncate table hotel_booking;
'''
with db_connections['hotel'].cursor() as cursor:
    try:
        cursor.execute(query)
        db_connections['hotel'].commit()
        print('hotel_booking tables created in hotel database')
    except Exception as e:
        print(e)
        db_connections['hotel'].rollback()

hotel_booking tables created in hotel database


In [6]:
query = '''
drop table if exists account;
create table account(
    account_id serial primary key,
    client_name varchar(255),
    deposit int check (deposit >= 0)
);
truncate table account;
insert into account(client_name, deposit) 
values 
('Dmytro Domin', 100)
'''
with db_connections['account'].cursor() as cursor:
    try:
        cursor.execute(query)
        db_connections['account'].commit()
        print('account tables created in account database, user added')
    except Exception as e:
        print(e)
        db_connections['account'].rollback()

account tables created in account database, user added


In [7]:
for connection in db_connections.values():
    connection.close()

### Working with flight and hotel databases with two phase commit 

In [8]:
def add_reservation(flight_record, hotel_record, flight_connection, hotel_connection):
    flight_query = '''
        insert into flight_booking(booking_id, client_name, flight_number, airpot_of_departure, destination_airport, date)
        values
        ({}, '{}', '{}', '{}', '{}', {});
    '''.format(*flight_record)
    hotel_query = '''
        insert into hotel_booking(booking_id, client_name, hotel_name, check_in_date, check_out_date)
        values
        ({}, '{}', '{}', {}, {});
    '''.format(*hotel_record)
    
    
    transaction_id = flight_connection.xid(1,'abc','de')
    
    with flight_connection.cursor() as flight_cursor, hotel_connection.cursor() as hotel_cursor:
        try:
            flight_connection.tpc_begin(transaction_id)
            hotel_connection.tpc_begin(transaction_id)
            flight_cursor.execute(flight_query)
            hotel_cursor.execute(hotel_query)
            flight_connection.tpc_prepare()
            hotel_connection.tpc_prepare()
            flight_connection.tpc_commit()
            hotel_connection.tpc_commit()
            return transaction_id, True
        except Exception as e:
            print(e)
            flight_connection.tpc_rollback()
            hotel_connection.tpc_rollback()
            return transaction_id, False

In [9]:
flight_bookings_to_insert = [
    (1, 'Ivan Baran', 'FR3069', 'KBP', 'BCN', 20220115),
    (2, 'Stepan Baran', 'FR7262', 'KRK', 'KBP', 20220115),
    (3, 'Petro Baran', 'PQ482', 'DXB', 'KBP', 20220115),
    (4, 'Mykola Baran', 'W66286', 'TLL', 'IEV', 20220115),
    (5, 'Fedir Baran', 'W66127 ', 'IEV', 'VIE', 20220115),
]

hotel_bookings_to_insert = [
    (1, 'Ivan Baran', 'Radisson', 20220115, 20220121),
    (2, 'Stepan Baran', 'Premier', 20220115, 20220121),
    (3, 'Petro Baran', 'Hiltons', 20220115, 20220121),
    (4, 'Mykola Baran', 'U Stepana', 20220115, 20220121),
    (5, 'Fedir Baran', 'Mozzart Pallace ', 20220115, 20220121),
]

try:
    connections = db_connections = {
    'flight': psycopg2.connect(**db_settings['flight']),
    'hotel': psycopg2.connect(**db_settings['hotel']),
    }

    # Happy Path
    for item in db_connections.items():
            print(item, f'version: {item[1].server_version}')

    for flight_booking, hotel_booking in zip (flight_bookings_to_insert, hotel_bookings_to_insert):
        transaction_id, transaction_commited = add_reservation(
            flight_booking,
            hotel_booking, 
            connections['flight'], 
            connections['hotel']
        )
        print(f'Transaction {transaction_id} done: {transaction_commited}')

    # Unhappy Path
    print(
        add_reservation(
            flight_booking,
            hotel_booking, 
            connections['flight'], 
            connections['hotel']
        )
    )
        
finally:
    for connection in connections.values():
        connection.close()

('flight', <connection object at 0x00000215325B1370; dsn: 'user=postgres password=xxx dbname=postgres host=localhost port=5001', closed: 0>) version: 110014
('hotel', <connection object at 0x00000215325B1040; dsn: 'user=postgres password=xxx dbname=postgres host=localhost port=5002', closed: 0>) version: 110014
Transaction 1_YWJj_ZGU= done: True
Transaction 1_YWJj_ZGU= done: True
Transaction 1_YWJj_ZGU= done: True
Transaction 1_YWJj_ZGU= done: True
Transaction 1_YWJj_ZGU= done: True
duplicate key value violates unique constraint "flight_booking_pkey"
DETAIL:  Key (booking_id)=(5) already exists.

(<Xid: (1, 'abc', 'de')>, False)


In [ ]:
for connection in connections.values():
    connection.close()